In [1]:
%config IPCompleter.greedy=True

In [2]:
import os
import pandas as pd

data_dir = "../data/"
datasets_fn = [
    (data_dir + file)
    for file in os.listdir(data_dir)
]

datasets = [
    pd.read_csv(file_name)
    for file_name in datasets_fn
]

datasets[0]

,transactionid,commitdate,ns,nm,nf,entropy,la,ld,lt,fix,ndev,pd,npt,exp,rexp,sexp,bug
0,3,2001/12/12 17:41,1,1,3,0.579380,0.093620,0.000000,480.666667,1,14,596,0.666667,143,133.500000,129,1
1,7,1999/10/12 12:57,1,1,1,0.000000,0.000000,0.000000,398.000000,1,1,0,1.000000,140,140.000000,137,1
2,8,2002/5/15 16:55,3,3,52,0.739279,0.183477,0.208913,283.519231,0,23,15836,0.750000,984,818.650000,978,0
3,9,2002/1/21 15:37,1,1,8,0.685328,0.016039,0.012880,514.375000,1,21,1281,1.000000,579,479.250000,550,0
4,10,2001/12/19 16:44,2,2,38,0.769776,0.091829,0.072746,366.815789,1,21,6565,0.763158,413,313.250000,405,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4615,20762,2002/7/26 16:44,1,1,1,0.000000,0.025698,0.039106,895.000000,1,21,0,1.000000,219,190.083333,212,1
4616,20840,2002/11/13 3:29,1,1,2,0.918296,0.005450,0.002725,183.500000,1,18,21,1.000000,355,252.833333,335,1
4617,20936,2005/9/27 12:54,1,1,1,0.000000,0.000000,0.000000,2.000000,1,2,0,1.000000,961,239.861905,3,0
4618,20937,2005/9/27 13:03,1,1,1,0.000000,21.000000,0.000000,2.000000,1,2,0,1.000000,615,464.500000,3,0


In [3]:
from sklearn.model_selection import KFold

kf_splitter = KFold(n_splits=10, shuffle=False, random_state=None)

def split_train_data(dataset):
    """Returns all the experiments with each experiment represented
    as a tuple: (training data, testing data)"""
    return [
        (dataset.iloc[train_indicies], dataset.iloc[test_indicies])
        for train_indicies, test_indicies in kf_splitter.split(dataset)
    ]

# Contains (train, test) for all data sets
splitted_data = [
    split_train_data(data)
    for data in datasets
]

data_train, data_test = splitted_data[0][0]
data_train

,transactionid,commitdate,ns,nm,nf,entropy,la,ld,lt,fix,ndev,pd,npt,exp,rexp,sexp,bug
462,1012,2002/7/6 5:33,1,1,1,0.000000,0.021978,0.000000,182.000000,1,20,3,1.000000,7,7.000000,7,1
463,1014,2002/6/14 12:56,1,1,1,0.000000,0.005525,0.005525,181.000000,1,19,37,1.000000,88,68.000000,86,0
464,1015,2002/5/8 17:25,1,1,1,0.000000,0.005525,0.000000,181.000000,0,19,0,1.000000,180,156.333333,173,0
465,1016,2002/5/8 15:02,1,1,1,0.000000,0.005556,0.011111,180.000000,1,19,14,1.000000,179,155.333333,172,0
466,1017,2002/4/24 3:24,2,2,28,0.934459,0.021663,0.029374,194.535714,1,22,974,0.785714,118,94.333333,118,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4615,20762,2002/7/26 16:44,1,1,1,0.000000,0.025698,0.039106,895.000000,1,21,0,1.000000,219,190.083333,212,1
4616,20840,2002/11/13 3:29,1,1,2,0.918296,0.005450,0.002725,183.500000,1,18,21,1.000000,355,252.833333,335,1
4617,20936,2005/9/27 12:54,1,1,1,0.000000,0.000000,0.000000,2.000000,1,2,0,1.000000,961,239.861905,3,0
4618,20937,2005/9/27 13:03,1,1,1,0.000000,21.000000,0.000000,2.000000,1,2,0,1.000000,615,464.500000,3,0


In [50]:
from sklearn.preprocessing import StandardScaler

def split_input_output(data):
    """Split the data into the information and the answer"""
    X = data
    X = X.drop('bug', axis=1)
    X = X.drop('transactionid', axis=1)
    X = X.drop('commitdate', axis=1)
    Y = data['bug']
    return (X, Y)

def scale_data_pair(train_test_data):
    
    train, test = train_test_data
    train_x, train_y = train
    test_x, test_y = test
    
    feature_scaler = StandardScaler()
    train_x = df = pd.DataFrame(feature_scaler.fit_transform(train_x.values), columns=train_x.columns, index=train_x.index)
    test_x = df = pd.DataFrame(feature_scaler.transform(test_x.values), columns=test_x.columns, index=test_x.index)
    
    train = (train_x, train_y)
    test = (test_x, test_y)
    
    return (train, test)

input_output_data = [
    [
        scale_data_pair((split_input_output(train_data), split_input_output(test_data)))
        for train_data, test_data in dataset
    ]
    for dataset in splitted_data
]

# First dataset, First experiment, Training data, data for training
input_output_data[0][0][0][0]

,ns,nm,nf,entropy,la,ld,lt,fix,ndev,pd,npt,exp,rexp,sexp
462,1,1,1,0.000000,0.021978,0.000000,182.000000,1,20,3,1.000000,7,7.000000,7
463,1,1,1,0.000000,0.005525,0.005525,181.000000,1,19,37,1.000000,88,68.000000,86
464,1,1,1,0.000000,0.005525,0.000000,181.000000,0,19,0,1.000000,180,156.333333,173
465,1,1,1,0.000000,0.005556,0.011111,180.000000,1,19,14,1.000000,179,155.333333,172
466,2,2,28,0.934459,0.021663,0.029374,194.535714,1,22,974,0.785714,118,94.333333,118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4615,1,1,1,0.000000,0.025698,0.039106,895.000000,1,21,0,1.000000,219,190.083333,212
4616,1,1,2,0.918296,0.005450,0.002725,183.500000,1,18,21,1.000000,355,252.833333,335
4617,1,1,1,0.000000,0.000000,0.000000,2.000000,1,2,0,1.000000,961,239.861905,3
4618,1,1,1,0.000000,21.000000,0.000000,2.000000,1,2,0,1.000000,615,464.500000,3


In [51]:
from sklearn import ensemble
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.evaluate import paired_ttest_kfold_cv
from sklearn.metrics import roc_auc_score, accuracy_score
import numpy as np
from scipy import stats


params = {
    'n_estimators': 1200,
    'max_depth': 3,
    'subsample': 0.5,
    'learning_rate': 0.01,
    'min_samples_leaf': 1,
    'random_state': 3
}

# First dataset, First experiment, Training data, data for training
# input_output_data[0][0][0][0]

for ds_num, dataset in enumerate(input_output_data):
    p_1_1 = 0.0
    # Initialize a place holder for the variance estimate
    s_sqr = 0.0
    # Initialize scores list for both classifiers
    scores_1 = []
    scores_2 = []
    diff_scores = []
    p_i = np.zeros(10)
    i_s = 0
    
    print("Fold,\tAccuracy Difference")
    
    for exp_num, experiment in enumerate(dataset):
        training, testing = experiment
        
        x_train = training[0]
        y_train = training[1]
        
        x_test = testing[0]
        y_test = testing[1]
        
        clf1 = ensemble.GradientBoostingClassifier(**params)
        clf2 = KNeighborsClassifier(n_neighbors=2)
        
        clf1.fit(x_train, y_train)
        clf2.fit(x_train, y_train)
        
        preds_1 = clf1.predict_proba(x_test)[:, 1]
        score_1 = roc_auc_score(y_test, preds_1)
        
        preds_2 = clf2.predict_proba(x_test)[:, 1]
        score_2 = roc_auc_score(y_test, preds_2)
        
        # keep score history for mean and stdev calculation
        scores_1.append(score_1)
        scores_2.append(score_2)
        
        diff_scores.append(score_1 - score_2)
        
        i_f = exp_num
        p_i[i_f] = score_1 - score_2
        
        print("%2d,\t%.6f" % (i_f + 1, p_i[i_f]))
    
    print("\n\nClassifier,\tMean,\tStdev")
    print("CLF,\t%.6f,\t%.6f" % (np.mean(scores_1), np.std(scores_1)))
    print("KNN,\t%.6f,\t%.6f" % (np.mean(scores_2), np.std(scores_2)))
    print("Delta,\t%.6f,\t%.6f" % (np.mean(diff_scores), np.std(diff_scores)))
    
    print("\n")
    
    n_splits = len(dataset)
    centered_diff = np.array(diff_scores) - np.mean(diff_scores)
    t = np.mean(diff_scores) * (n_splits ** .5) / (np.sqrt(np.sum(centered_diff ** 2) / (n_splits - 1)))
    print("t = %.6f" % (t))

    # Getting the t value
    ci = 95 # Confidence level
    n = 10  # Degrees of freedom = number of experiments
    t_threshold = stats.t.ppf(1- ((100-ci)/2/100), n-1)
    print("t_threshold = %.6f" % (t_threshold))
    
    if t > t_threshold:
        print("t > t_threshold => significant difference in performance")
    else:
        print("t < t_threshold => no significant difference in performance")
    break


Fold,	Accuracy Difference
 1,	0.276761
 2,	0.295159
 3,	0.252296
 4,	0.223836
 5,	0.241556
 6,	0.213199
 7,	0.208858
 8,	0.248878
 9,	0.220035
10,	0.235261


Classifier,	Mean,	Stdev
CLF,	0.821218,	0.029618
KNN,	0.579634,	0.040044
Delta,	0.241584,	0.026404


t = 27.448597
t_threshold = 2.262157
t > t_threshold => significant difference in performance
